In [25]:
import os
import scipy.io
import numpy as np
from scipy import signal

data_path = "/kaggle/input/control1"
lowcut = 0.4 
highcut = 100 
fs_original = 500
fs_new = 250

control_data = []
control_path = os.path.join(data_path, 'Control')

for foldername in os.listdir(control_path):
    subfolder_path = os.path.join(control_path, foldername)
    control_data.append(subfolder_path)

Control30 = []

for control_data_path in control_data:
    n_epochs = 70 
    start_epoch = 1
    end_epoch = 110

    control_arr = []

    for i in range(start_epoch, end_epoch+1):
        epoch_path = os.path.join(control_data_path, f"trial{i}.mat")
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]    
        #print(mat_data1.shape)
        # Downsampling
        num_samples_original = mat_data1.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(mat_data1, num_samples_new, axis=-1)
        #print(downsampled_data.shape)
        
        control_arr.append(downsampled_data)
        Control30.append(downsampled_data)

Control30 = np.array(Control30)

print(Control30.shape)


(3300, 60, 1000)


In [27]:
import os
import scipy.io
import numpy as np
from scipy import signal
data_path = "/kaggle/input/concussed1"
lowcut = 0.4 
highcut = 100 
fs_original = 500
fs_new = 250  

concussed_data = []
concussed_path = os.path.join(data_path, 'Concussed')
for foldername in os.listdir(concussed_path):
    subfolder_path = os.path.join(concussed_path, foldername)
    concussed_data.append(subfolder_path)

Concussed52 = []
for concussed_data_path in concussed_data:
    n_epochs = 70
    start_epoch = 1
    end_epoch = 110
    concussed_arr = []

    for i in range(start_epoch, end_epoch+1):
        epoch_path = f"{concussed_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
        num_samples_original = mat_data1.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(mat_data1, num_samples_new, axis=-1)
        
        concussed_arr.append(np.array(downsampled_data))
        
        Concussed52.append(downsampled_data)

Concussed52 = np.array(Concussed52)

print( Concussed52.shape)


(5390, 60, 1000)


In [28]:
dataset = np.concatenate((Control30, Concussed52), axis=0)
labels = np.concatenate((np.zeros(Control30.shape[0]), np.ones(Concussed52.shape[0])))

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=42)
mean = np.mean(X_train)
std = np.std(X_train)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

model = Sequential()
model.add(Conv1D(32, 3, activation='relu', input_shape=(60, 1000)))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [32]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
218/218 [==============================] - 5s 24ms/step - loss: 0.1132 - accuracy: 0.9600
Epoch 2/10
218/218 [==============================] - 5s 22ms/step - loss: 0.0540 - accuracy: 0.9855
Epoch 3/10
218/218 [==============================] - 5s 25ms/step - loss: 0.0522 - accuracy: 0.9856
Epoch 4/10
218/218 [==============================] - 5s 24ms/step - loss: 0.0416 - accuracy: 0.9882
Epoch 5/10
218/218 [==============================] - 5s 23ms/step - loss: 0.0364 - accuracy: 0.9919
Epoch 6/10
218/218 [==============================] - 5s 24ms/step - loss: 0.3036 - accuracy: 0.9085
Epoch 7/10
218/218 [==============================] - 5s 23ms/step - loss: 0.1008 - accuracy: 0.9632
Epoch 8/10
218/218 [==============================] - 5s 23ms/step - loss: 0.1098 - accuracy: 0.9675
Epoch 9/10
218/218 [==============================] - 5s 24ms/step - loss: 0.0612 - accuracy: 0.9787
Epoch 10/10
218/218 [==============================] - 5s 24ms/step - loss: 0.0303 - accura

In [33]:
_, accuracy = model.evaluate(X_test, y_test)
print("Accuracy: %.2f%%" % (accuracy * 100))

55/55 [==============================] - 1s 9ms/step - loss: 0.6686 - accuracy: 0.8533
Accuracy: 85.33%


In [34]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).flatten()
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

55/55 [==============================] - 1s 10ms/step
Confusion Matrix:
[[508 176]
 [ 79 975]]


In [35]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

         0.0       0.87      0.74      0.80       684
         1.0       0.85      0.93      0.88      1054

    accuracy                           0.85      1738
   macro avg       0.86      0.83      0.84      1738
weighted avg       0.85      0.85      0.85      1738



In [3]:
print(Control30.shape)
print(Control19.shape)
Control30 = np.concatenate((Control30, Control19), axis=0)
print(Control30.shape)

(30, 36, 60, 1000)
(19, 36, 60, 1000)
(49, 36, 60, 1000)


In [4]:
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import entropy

delta_band = (0.5, 4)
alpha_band = (8, 12)
beta_band = (12, 35)

all_control_power = []
scaler = MinMaxScaler()
epsilon = 1e-10

# Normalizing the features
for i in range(len(Control30)):
    control_data = Control30[i]
    control_power = []
    for epoch in control_data:
        epoch_power = []
        for channel_data in epoch:
            freq_spectrum = np.fft.fft(channel_data)
            power_spectrum = np.abs(freq_spectrum) ** 2

            alpha_power = np.sum(power_spectrum[(alpha_band[0] <= freq_spectrum) & (freq_spectrum <= alpha_band[1])])
            beta_power = np.sum(power_spectrum[(beta_band[0] <= freq_spectrum) & (freq_spectrum <= beta_band[1])])
            delta_power = np.sum(power_spectrum[(delta_band[0] <= freq_spectrum) & (freq_spectrum <= delta_band[1])])

            channel_min_value = np.min(channel_data)
            channel_max_value = np.max(channel_data)
            power_spectrum_adjusted = power_spectrum + epsilon
            entropy_values = entropy(power_spectrum_adjusted)
            values=np.array([alpha_power,beta_power,delta_power,entropy_values])
            rescaled_values = (values - values.min()) * (channel_max_value - channel_min_value) / (values.max() - values.min()) + channel_min_value
            concatenated_data = np.concatenate((channel_data, rescaled_values))
            epoch_power.append(concatenated_data.tolist())

        control_power.append(np.array(epoch_power))

    all_control_power.append(np.array(control_power))

all_control_power = np.array(all_control_power)
print(all_control_power.shape)
print(len(Control30))


(49, 36, 60, 1004)
49


In [5]:
# Control30_part1 = Control30[:, :, :, :500]
# Control30_part2 = Control30[:, :, :, :500]
# print("Shape of Control30_part1:", Control30_part1.shape)
# print("Shape of Control30_part2:", Control30_part2.shape)


In [6]:
# import numpy as np
# import json
# from sklearn.preprocessing import MinMaxScaler
# from scipy.stats import entropy

# delta_band = (0.5, 4)
# alpha_band = (8, 12)
# beta_band = (12, 35)

# all_control_power = []
# scaler = MinMaxScaler()
# epsilon = 1e-10

# # Divide the EEG data into two parts
# Control30_part1 = Control30[:, :, :, :500]
# Control30_part2 = Control30[:, :, :, :500]

# # Calculate features for Control30_part1
# all_control_power_part1 = []
# for i in range(len(Control30_part1)):
#     control_data = Control30_part1[i]
#     control_power = []
#     for epoch in control_data:
#         epoch_power = []
#         for channel_data in epoch:
#             freq_spectrum = np.fft.fft(channel_data)
#             power_spectrum = np.abs(freq_spectrum) ** 2

#             alpha_power = np.sum(power_spectrum[(alpha_band[0] <= freq_spectrum) & (freq_spectrum <= alpha_band[1])])
#             beta_power = np.sum(power_spectrum[(beta_band[0] <= freq_spectrum) & (freq_spectrum <= beta_band[1])])
#             delta_power = np.sum(power_spectrum[(delta_band[0] <= freq_spectrum) & (freq_spectrum <= delta_band[1])])

#             channel_min_value = np.min(channel_data)
#             channel_max_value = np.max(channel_data)

#             power_spectrum_adjusted = power_spectrum + epsilon
#             entropy_values = entropy(power_spectrum_adjusted)
#             values = np.array([alpha_power, beta_power, delta_power, entropy_values])
#             rescaled_values = (values - values.min()) * (channel_max_value - channel_min_value) / (values.max() - values.min()) + channel_min_value
#             concatenated_data = np.concatenate((channel_data, rescaled_values))

#             epoch_power.append(concatenated_data.tolist())

#         control_power.append(np.array(epoch_power))

#     all_control_power_part1.append(np.array(control_power))

# all_control_power_part1 = np.array(all_control_power_part1)
# print("Shape of all_control_power_part1:", all_control_power_part1.shape)

# # Calculate features for Control30_part2
# all_control_power_part2 = []
# for i in range(len(Control30_part2)):
#     control_data = Control30_part2[i]
#     control_power = []
#     for epoch in control_data:
#         epoch_power = []
#         for channel_data in epoch:
#             freq_spectrum = np.fft.fft(channel_data)
#             power_spectrum = np.abs(freq_spectrum) ** 2

#             alpha_power = np.sum(power_spectrum[(alpha_band[0] <= freq_spectrum) & (freq_spectrum <= alpha_band[1])])
#             beta_power = np.sum(power_spectrum[(beta_band[0] <= freq_spectrum) & (freq_spectrum <= beta_band[1])])
#             delta_power = np.sum(power_spectrum[(delta_band[0] <= freq_spectrum) & (freq_spectrum <= delta_band[1])])

#             channel_min_value = np.min(channel_data)
#             channel_max_value = np.max(channel_data)

#             power_spectrum_adjusted = power_spectrum + epsilon
#             entropy_values = entropy(power_spectrum_adjusted)
#             values = np.array([alpha_power, beta_power, delta_power, entropy_values])
#             rescaled_values = (values - values.min()) * (channel_max_value - channel_min_value) / (values.max() - values.min()) + channel_min_value
#             concatenated_data = np.concatenate((channel_data, rescaled_values))

#             epoch_power.append(concatenated_data.tolist())

#         control_power.append(np.array(epoch_power))

#     all_control_power_part2.append(np.array(control_power))

# all_control_power_part2 = np.array(all_control_power_part2)
# print("Shape of all_control_power_part2:", all_control_power_part2.shape)

# # Concatenate the features along the second axis
# Combined_data_Control = np.concatenate((all_control_power_part1, all_control_power_part2), axis=1)
# print("Shape of Combined_data:", Combined_data_Control.shape)


In [11]:
import os
import scipy.io
import numpy as np
from scipy import signal
data_path = "/kaggle/input/concussed1"
lowcut = 0.4 
highcut = 100 
fs_original = 500
fs_new = 250  

concussed_data = []
concussed_path = os.path.join(data_path, 'Concussed')
for foldername in os.listdir(concussed_path):
    subfolder_path = os.path.join(concussed_path, foldername)
    concussed_data.append(subfolder_path)

Concussed52 = []
for concussed_data_path in concussed_data:
    n_epochs = 70
    start_epoch = 35
    end_epoch = 70
    concussed_arr = []

    for i in range(start_epoch, end_epoch+1):
        epoch_path = f"{concussed_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
        num_samples_original = mat_data1.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(mat_data1, num_samples_new, axis=-1)
        
        concussed_arr.append(np.array(downsampled_data))
        
        Concussed52.append(downsampled_data)

Concussed52 = np.array(Concussed52)

print( Concussed52.shape)


(1764, 60, 1000)


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [17]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
72/72 [==============================] - 3s 25ms/step - loss: 0.7033 - accuracy: 0.6202
Epoch 2/10
72/72 [==============================] - 2s 23ms/step - loss: 0.5275 - accuracy: 0.7196
Epoch 3/10
72/72 [==============================] - 2s 24ms/step - loss: 0.4849 - accuracy: 0.7802
Epoch 4/10
72/72 [==============================] - 2s 24ms/step - loss: 0.4031 - accuracy: 0.8207
Epoch 5/10
72/72 [==============================] - 2s 23ms/step - loss: 0.3634 - accuracy: 0.8426
Epoch 6/10
72/72 [==============================] - 2s 22ms/step - loss: 0.2608 - accuracy: 0.9055
Epoch 7/10
72/72 [==============================] - 2s 22ms/step - loss: 0.1858 - accuracy: 0.9354
Epoch 8/10
72/72 [==============================] - 2s 25ms/step - loss: 0.1270 - accuracy: 0.9631
Epoch 9/10
72/72 [==============================] - 2s 25ms/step - loss: 0.0959 - accuracy: 0.9749
Epoch 10/10
72/72 [==============================] - 2s 24ms/step - loss: 0.0636 - accuracy: 0.9877


In [18]:
_, accuracy = model.evaluate(X_test, y_test)
print("Accuracy: %.2f%%" % (accuracy * 100))

18/18 [==============================] - 0s 9ms/step - loss: 0.6896 - accuracy: 0.7944
Accuracy: 79.44%


In [19]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).flatten()
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)


18/18 [==============================] - 0s 9ms/step


In [23]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

         0.0       0.75      0.69      0.72       217
         1.0       0.82      0.86      0.84       352

    accuracy                           0.79       569
   macro avg       0.78      0.77      0.78       569
weighted avg       0.79      0.79      0.79       569



In [8]:
# Concussed52_part1 = Concussed52[:, :, :, :500]
# Concussed52_part2 = Concussed52[:, :, :, :500]
# print("Shape of Concussed52_part1:", Concussed52_part1.shape)
# print("Shape of Concussed52_part2:", Concussed52_part2.shape)


In [9]:
# import numpy as np
# import json
# from sklearn.preprocessing import MinMaxScaler
# from scipy.stats import entropy

# delta_band = (0.5, 4)
# alpha_band = (8, 12)
# beta_band = (12, 35)

# all_control_power = []
# scaler = MinMaxScaler()
# epsilon = 1e-10

# # Divide the EEG data into two parts
# Concussed52_part1 = Concussed52[:, :, :, :500]
# Concussed52_part2 = Concussed52[:, :, :, :500]

# # Calculate features for Concussed52_part1
# all_concussed_power_part1 = []
# for i in range(len(Concussed52_part1)):
#     concussed_data = Concussed52_part1[i]
#     concussed_power = []
#     for epoch in concussed_data:
#         epoch_power = []
#         for channel_data in epoch:
#             freq_spectrum = np.fft.fft(channel_data)
#             power_spectrum = np.abs(freq_spectrum) ** 2

#             alpha_power = np.sum(power_spectrum[(alpha_band[0] <= freq_spectrum) & (freq_spectrum <= alpha_band[1])])
#             beta_power = np.sum(power_spectrum[(beta_band[0] <= freq_spectrum) & (freq_spectrum <= beta_band[1])])
#             delta_power = np.sum(power_spectrum[(delta_band[0] <= freq_spectrum) & (freq_spectrum <= delta_band[1])])

#             channel_min_value = np.min(channel_data)
#             channel_max_value = np.max(channel_data)

#             power_spectrum_adjusted = power_spectrum + epsilon
#             entropy_values = entropy(power_spectrum_adjusted)
#             values = np.array([alpha_power, beta_power, delta_power, entropy_values])
#             rescaled_values = (values - values.min()) * (channel_max_value - channel_min_value) / (values.max() - values.min()) + channel_min_value
#             concatenated_data = np.concatenate((channel_data, rescaled_values))

#             epoch_power.append(concatenated_data.tolist())

#         concussed_power.append(np.array(epoch_power))

#     all_concussed_power_part1.append(np.array(concussed_power))

# all_concussed_power_part1 = np.array(all_concussed_power_part1)
# print("Shape of all_concussed_power_part1:", all_concussed_power_part1.shape)

# # Calculate features for Concussed52_part2
# all_concussed_power_part2 = []
# for i in range(len(Concussed52_part2)):
#     concussed_data = Concussed52_part2[i]
#     concussed_power = []
#     for epoch in concussed_data:
#         epoch_power = []
#         for channel_data in epoch:
#             freq_spectrum = np.fft.fft(channel_data)
#             power_spectrum = np.abs(freq_spectrum) ** 2

#             alpha_power = np.sum(power_spectrum[(alpha_band[0] <= freq_spectrum) & (freq_spectrum <= alpha_band[1])])
#             beta_power = np.sum(power_spectrum[(beta_band[0] <= freq_spectrum) & (freq_spectrum <= beta_band[1])])
#             delta_power = np.sum(power_spectrum[(delta_band[0] <= freq_spectrum) & (freq_spectrum <= delta_band[1])])

#             channel_min_value = np.min(channel_data)
#             channel_max_value = np.max(channel_data)

#             power_spectrum_adjusted = power_spectrum + epsilon
#             entropy_values = entropy(power_spectrum_adjusted)
#             values = np.array([alpha_power, beta_power, delta_power, entropy_values])
#             rescaled_values = (values - values.min()) * (channel_max_value - channel_min_value) / (values.max() - values.min()) + channel_min_value
#             concatenated_data = np.concatenate((channel_data, rescaled_values))

#             epoch_power.append(concatenated_data.tolist())

#         concussed_power.append(np.array(epoch_power))

#     all_concussed_power_part2.append(np.array(concussed_power))

# all_concussed_power_part2 = np.array(all_concussed_power_part2)
# print("Shape of all_concussed_power_part2:", all_concussed_power_part2.shape)

# # Concatenate the features from both parts along the second axis
# Combined_data_Concussed = np.concatenate((all_concussed_power_part1, all_concussed_power_part2), axis=1)
# print("Shape of Combined_data:", Combined_data_Concussed.shape)


In [10]:
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import entropy

delta_band = (0.5, 4)
alpha_band = (8, 12)
beta_band = (12, 35)

all_concussed_power = []
scaler = MinMaxScaler()
epsilon = 1e-10

for i in range(len(Concussed52)):
    concussed_data = Concussed52[i]
    concussed_power = []
    for epoch in concussed_data:
        epoch_power = []
        for channel_data in epoch:
            freq_spectrum = np.fft.fft(channel_data)
            power_spectrum = np.abs(freq_spectrum) ** 2

            alpha_power = np.sum(power_spectrum[(alpha_band[0] <= freq_spectrum) & (freq_spectrum <= alpha_band[1])])
            beta_power = np.sum(power_spectrum[(beta_band[0] <= freq_spectrum) & (freq_spectrum <= beta_band[1])])
            delta_power = np.sum(power_spectrum[(delta_band[0] <= freq_spectrum) & (freq_spectrum <= delta_band[1])])

            channel_min_value = np.min(channel_data)
            channel_max_value = np.max(channel_data)
            power_spectrum_adjusted = power_spectrum + epsilon
            entropy_values = entropy(power_spectrum_adjusted)
            values=np.array([alpha_power,beta_power,delta_power,entropy_values])
            rescaled_values = (values - values.min()) * (channel_max_value - channel_min_value) / (values.max() - values.min()) + channel_min_value
            concatenated_data = np.concatenate((channel_data, rescaled_values))
            epoch_power.append(concatenated_data.tolist())

        concussed_power.append(np.array(epoch_power))

    all_concussed_power.append(np.array(concussed_power))

all_concussed_power = np.array(all_concussed_power)
print(all_concussed_power.shape)
print(len(Concussed52))

(49, 36, 60, 1004)
49


In [11]:
from sklearn.model_selection import train_test_split

x_control_train, x_control_test, y_control_train, y_control_test = train_test_split(all_control_power, np.zeros((49,)), test_size=0.2, random_state=42)
x_control_train, x_control_val, y_control_train, y_control_val = train_test_split(x_control_train, y_control_train, test_size=0.2, random_state=42)

x_concussed_train, x_concussed_test, y_concussed_train, y_concussed_test = train_test_split(all_concussed_power, np.ones((49,)), test_size=0.2, random_state=42)
x_concussed_train, x_concussed_val, y_concussed_train, y_concussed_val = train_test_split(x_concussed_train, y_concussed_train, test_size=0.2, random_state=42)

# Combine the control and concussed data
x_train = np.concatenate((x_control_train, x_concussed_train), axis=0)
y_train = np.concatenate((y_control_train, y_concussed_train), axis=0)
x_val = np.concatenate((x_control_val, x_concussed_val), axis=0)
y_val = np.concatenate((y_control_val, y_concussed_val), axis=0)
x_test = np.concatenate((x_control_test, x_concussed_test), axis=0)
y_test = np.concatenate((y_control_test, y_concussed_test), axis=0)

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_normalized = scaler.fit_transform(x_train.reshape(-1, 1004)).reshape(x_train.shape)
x_val_normalized = scaler.transform(x_val.reshape(-1, 1004)).reshape(x_val.shape)
x_test_normalized = scaler.transform(x_test.reshape(-1, 1004)).reshape(x_test.shape)

In [13]:
from keras.utils import to_categorical

y_train_categorical = to_categorical(y_train)
y_val_categorical = to_categorical(y_val)
y_test_categorical = to_categorical(y_test)


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Epoch 1/50
1/1 [==============================] - 6s 6s/step - loss: 0.6977 - accuracy: 0.4839 - val_loss: 0.7280 - val_accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 4s 4s/step - loss: 0.6866 - accuracy: 0.6129 - val_loss: 0.7247 - val_accuracy: 0.5000
Epoch 3/50
1/1 [==============================] - 4s 4s/step - loss: 0.8293 - accuracy: 0.5968 - val_loss: 0.7216 - val_accuracy: 0.5000
Epoch 4/50
1/1 [==============================] - 4s 4s/step - loss: 1.3549 - accuracy: 0.5000 - val_loss: 0.7206 - val_accuracy: 0.5000
Epoch 5/50
1/1 [==============================] - 4s 4s/step - loss: 0.6764 - accuracy: 0.4516 - val_loss: 0.7196 - val_accuracy: 0.5000
Epoch 6/50
1/1 [==============================] - 4s 4s/step - loss: 0.7185 - accuracy: 0.5968 - val_loss: 0.7182 - val_accuracy: 0.5000
Epoch 7/50
1/1 [==============================] - 4s 4s/step - loss: 0.6707 - accuracy: 0.5645 - val_loss: 0.7167 - val_accuracy: 0.5000
Epoch 8/50
1/1 [=========================

In [29]:
loss, accuracy = model.evaluate(x_test_normalized, y_test_categorical)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

1/1 [==============================] - 0s 416ms/step - loss: 0.6499 - accuracy: 0.5000
Test loss: 0.64994215965271
Test accuracy: 0.5


In [16]:
fggj

NameError: name 'fggj' is not defined

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
data_combined = np.concatenate((all_control_power, all_concussed_power), axis=0)

control_labels = np.zeros(all_control_power.shape[0])
concussed_labels = np.ones(all_concussed_power.shape[0])
labels_combined = np.concatenate((control_labels, concussed_labels))

x_train_val, x_test, y_train_val, y_test = train_test_split(data_combined, labels_combined, test_size=0.2, stratify=labels_combined, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.2, stratify=y_train_val, random_state=42)
scaler = MinMaxScaler()

x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_val_flat = x_val.reshape(x_val.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)
x_train_normalized = scaler.fit_transform(x_train_flat)
x_val_normalized = scaler.transform(x_val_flat)
x_test_normalized = scaler.transform(x_test_flat)

x_train_normalized = x_train_normalized.reshape(x_train.shape)
x_val_normalized = x_val_normalized.reshape(x_val.shape)
x_test_normalized = x_test_normalized.reshape(x_test.shape)
print("x_train shape:", x_train_normalized.shape)
print("y_train shape:", y_train.shape)
print("x_val shape:", x_val_normalized.shape)
print("y_val shape:", y_val.shape)
print("x_test shape:", x_test_normalized.shape)
print("y_test shape:", y_test.shape)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
from keras import regularizers

x_train_reshaped = x_train_normalized.reshape(x_train_normalized.shape[0], -1)
x_test_reshaped = x_test_normalized.reshape(x_test_normalized.shape[0], -1)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_reshaped)
x_test_scaled = scaler.transform(x_test_reshaped)

y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
x_train_scaled = x_train_scaled.reshape(x_train_normalized.shape[0], x_train_normalized.shape[1], -1)
x_test_scaled = x_test_scaled.reshape(x_test_normalized.shape[0], x_test_normalized.shape[1], -1)

model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)),
                        input_shape=(x_train_normalized.shape[1], x_train_scaled.shape[2])))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=256, return_sequences=True, kernel_regularizer=regularizers.l2(0.01))))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=128, return_sequences=True, kernel_regularizer=regularizers.l2(0.01))))
model.add(Dropout(0.5))
model.add(Bidirectional(GRU(units=64, return_sequences=True, kernel_regularizer=regularizers.l2(0.01))))
model.add(Dropout(0.5))
model.add(Bidirectional(GRU(units=32, kernel_regularizer=regularizers.l2(0.01))))
model.add(Dropout(0.5))
model.add(Dense(units=16, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(units=2, activation='softmax'))

learning_rate = 0.0006
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
history = model.fit(x_train_scaled, y_train_categorical, validation_split=0.2,
                    epochs=200, batch_size=64, callbacks=[early_stopping], verbose=0)

loss, accuracy = model.evaluate(x_test_scaled, y_test_categorical)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

test_predictions = model.predict(x_test_scaled)
test_predicted_labels = np.argmax(test_predictions, axis=1)
test_true_labels = np.argmax(y_test_categorical, axis=1)

cm_test = confusion_matrix(test_true_labels, test_predicted_labels)
print('Confusion Matrix (Test):')
print(cm_test)

class_report_test = classification_report(test_true_labels, test_predicted_labels)
print('Classification Report (Test):')
print(class_report_test)


In [ ]:
dataset = np.concatenate((Combined_data_Control, Combined_data_Concussed), axis=0)
labels = np.concatenate((np.zeros(len(Combined_data_Control)), np.ones(len(Combined_data_Concussed))))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(dataset, labels, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model architecture
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(140,60,504)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
callback_list = EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True)


In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
    metrics=['accuracy']
)


In [ ]:
from sklearn.model_selection import KFold
X =dataset
y = labels 
Epoch = 10
batchSize = 256

# Define the number of folds for cross-validation
k_folds = 3
kf = KFold(n_splits=k_folds, shuffle=True)

train_accuracies = []
val_accuracies = []

true_labels = []
predicted_labels = []


# cross-validation
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    history = model.fit(
    X_train,
    y_train,
    epochs=Epoch,
    batch_size=batchSize,
    validation_data=(X_val, y_val),
    callbacks=callback_list,
    verbose=1)

    # Evaluate the model on training and validation data
    train_loss, train_accuracy = model.evaluate(X_train, y_train)
    val_loss, val_accuracy = model.evaluate(X_val, y_val)

    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)
    
    val_predictions = model.predict(X_val)
    val_predictions = np.argmax(val_predictions, axis=1)
    true_labels.extend(y_val)
    predicted_labels.extend(val_predictions)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Convert the true labels and predicted labels to numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Calculate the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(cm)

# Calculate the classification report
report = classification_report(true_labels, predicted_labels)
print("Classification Report:")
print(report)


In [ ]:
New One

In [ ]:
import os
import scipy.io
import numpy as np
from scipy import signal
data_path = "/kaggle/input/control1"
lowcut = 0.4 
highcut = 100 
fs_original = 500
fs_new = 250

control_data = []
control_path = os.path.join(data_path, 'Control')

for foldername in os.listdir(control_path):
    subfolder_path = os.path.join(control_path, foldername)
    control_data.append(subfolder_path)

Control30 = []

for control_data_path in control_data:
    n_epochs = 70 
    start_epoch = 35
    end_epoch = 70
    control_arr = []

    for i in range(start_epoch, end_epoch+1):
        epoch_path = os.path.join(control_data_path, f"trial{i}.mat")
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]        
        # Downsampling
        num_samples_original = mat_data1.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(mat_data1, num_samples_new, axis=-1)
        
        control_arr.append(np.array(downsampled_data))
        
    Control30.append(np.array(control_arr))

Control30 = np.array(Control30)
print(Control30[0].shape)

In [ ]:
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import entropy

delta_band = (0.5, 4)
alpha_band = (8, 12)
beta_band = (12, 35)

all_control_power = []
scaler = MinMaxScaler()
epsilon = 1e-10

# Normalizing the features
for i in range(len(Control30)):
    control_data = Control30[i]
    control_power = []
    for epoch in control_data:
        epoch_power = []
        for channel_data in epoch:
            freq_spectrum = np.fft.fft(channel_data)
            power_spectrum = np.abs(freq_spectrum) ** 2

            alpha_power = np.sum(power_spectrum[(alpha_band[0] <= freq_spectrum) & (freq_spectrum <= alpha_band[1])])
            beta_power = np.sum(power_spectrum[(beta_band[0] <= freq_spectrum) & (freq_spectrum <= beta_band[1])])
            delta_power = np.sum(power_spectrum[(delta_band[0] <= freq_spectrum) & (freq_spectrum <= delta_band[1])])

            channel_min_value = np.min(channel_data)
            channel_max_value = np.max(channel_data)
            power_spectrum_adjusted = power_spectrum + epsilon
            entropy_values = entropy(power_spectrum_adjusted)
            values=np.array([alpha_power,beta_power,delta_power,entropy_values])
            rescaled_values = (values - values.min()) * (channel_max_value - channel_min_value) / (values.max() - values.min()) + channel_min_value
            concatenated_data = np.concatenate((channel_data, rescaled_values))
            epoch_power.append(concatenated_data.tolist())

        control_power.append(np.array(epoch_power))

    all_control_power.append(np.array(control_power))

all_control_power = np.array(all_control_power)
print(all_control_power.shape)
print(len(Control30))


In [ ]:
import os
import scipy.io
import numpy as np
from scipy import signal
data_path = "/kaggle/input/concussed1"
lowcut = 0.4 
highcut = 100 
fs_original = 500
fs_new = 250  

concussed_data = []
concussed_path = os.path.join(data_path, 'Concussed')
for foldername in os.listdir(concussed_path):
    subfolder_path = os.path.join(concussed_path, foldername)
    concussed_data.append(subfolder_path)

Concussed52 = []
for concussed_data_path in concussed_data:
    n_epochs = 70
    start_epoch = 35
    end_epoch = 70
    concussed_arr = []

    for i in range(start_epoch, end_epoch+1):
        epoch_path = f"{concussed_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
        num_samples_original = mat_data1.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(mat_data1, num_samples_new, axis=-1)
        
        concussed_arr.append(np.array(downsampled_data))
        
    Concussed52.append(np.array(concussed_arr))

Concussed52 = np.array(Concussed52)
print( Concussed52[0].shape)

In [ ]:
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import entropy

delta_band = (0.5, 4)
alpha_band = (8, 12)
beta_band = (12, 35)

all_concussed_power = []
scaler = MinMaxScaler()
epsilon = 1e-10

for i in range(len(Concussed52)):
    concussed_data = Concussed52[i]
    concussed_power = []
    for epoch in concussed_data:
        epoch_power = []
        for channel_data in epoch:
            freq_spectrum = np.fft.fft(channel_data)
            power_spectrum = np.abs(freq_spectrum) ** 2

            alpha_power = np.sum(power_spectrum[(alpha_band[0] <= freq_spectrum) & (freq_spectrum <= alpha_band[1])])
            beta_power = np.sum(power_spectrum[(beta_band[0] <= freq_spectrum) & (freq_spectrum <= beta_band[1])])
            delta_power = np.sum(power_spectrum[(delta_band[0] <= freq_spectrum) & (freq_spectrum <= delta_band[1])])

            channel_min_value = np.min(channel_data)
            channel_max_value = np.max(channel_data)
            power_spectrum_adjusted = power_spectrum + epsilon
            entropy_values = entropy(power_spectrum_adjusted)
            values=np.array([alpha_power,beta_power,delta_power,entropy_values])
            rescaled_values = (values - values.min()) * (channel_max_value - channel_min_value) / (values.max() - values.min()) + channel_min_value
            concatenated_data = np.concatenate((channel_data, rescaled_values))
            epoch_power.append(concatenated_data.tolist())

        concussed_power.append(np.array(epoch_power))

    all_concussed_power.append(np.array(concussed_power))

all_concussed_power = np.array(all_concussed_power)
print(all_concussed_power.shape)
print(len(Concussed52))


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
data_combined = np.concatenate((all_control_power, all_concussed_power), axis=0)

control_labels = np.zeros(all_control_power.shape[0])
concussed_labels = np.ones(all_concussed_power.shape[0])
labels_combined = np.concatenate((control_labels, concussed_labels))

x_train_val, x_test, y_train_val, y_test = train_test_split(data_combined, labels_combined, test_size=0.2, stratify=labels_combined, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.2, stratify=y_train_val, random_state=42)
scaler = MinMaxScaler()

x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_val_flat = x_val.reshape(x_val.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)
x_train_normalized = scaler.fit_transform(x_train_flat)
x_val_normalized = scaler.transform(x_val_flat)
x_test_normalized = scaler.transform(x_test_flat)

x_train_normalized = x_train_normalized.reshape(x_train.shape)
x_val_normalized = x_val_normalized.reshape(x_val.shape)
x_test_normalized = x_test_normalized.reshape(x_test.shape)
print("x_train shape:", x_train_normalized.shape)
print("y_train shape:", y_train.shape)
print("x_val shape:", x_val_normalized.shape)
print("y_val shape:", y_val.shape)
print("x_test shape:", x_test_normalized.shape)
print("y_test shape:", y_test.shape)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
from keras import regularizers

x_train_reshaped = x_train_normalized.reshape(x_train_normalized.shape[0], -1)
x_test_reshaped = x_test_normalized.reshape(x_test_normalized.shape[0], -1)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_reshaped)
x_test_scaled = scaler.transform(x_test_reshaped)
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

x_train_scaled = x_train_scaled.reshape(x_train_normalized.shape[0], x_train_normalized.shape[1], -1)
x_test_scaled = x_test_scaled.reshape(x_test_normalized.shape[0], x_test_normalized.shape[1], -1)
n_splits = 5

kf = KFold(n_splits=n_splits, shuffle=True)

# Lists to store evaluation results
loss_scores = []
accuracy_scores = []
all_predictions = []
all_true_labels = []

# Perform k-fold cross-validation
for train_index, val_index in kf.split(x_train_scaled):
    # Split the data into training and validation sets for this fold
    x_train_fold, x_val_fold = x_train_scaled[train_index], x_train_scaled[val_index]
    y_train_fold, y_val_fold = y_train_categorical[train_index], y_train_categorical[val_index]

    # Build the model architecture with regularization
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)),
                            input_shape=(x_train_normalized.shape[1], x_train_scaled.shape[2])))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(units=256, return_sequences=True, kernel_regularizer=regularizers.l2(0.01))))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(units=128, return_sequences=True, kernel_regularizer=regularizers.l2(0.01))))
    model.add(Dropout(0.5))
    model.add(Bidirectional(GRU(units=64, return_sequences=True, kernel_regularizer=regularizers.l2(0.01))))
    model.add(Dropout(0.5))
    model.add(Bidirectional(GRU(units=32, kernel_regularizer=regularizers.l2(0.01))))
    model.add(Dropout(0.5))
    model.add(Dense(units=16, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dense(units=2, activation='softmax'))

    # Set the learning rate
    learning_rate = 0.0006
    optimizer = Adam(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Define early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # Train the model with early stopping
    history = model.fit(x_train_fold, y_train_fold, validation_data=(x_val_fold, y_val_fold),
                        epochs=200, batch_size=128, callbacks=[early_stopping], verbose=0)

    # Evaluate the model on the validation set
    loss, accuracy = model.evaluate(x_val_fold, y_val_fold)
    loss_scores.append(loss)
    accuracy_scores.append(accuracy)
    
    val_predictions = model.predict(x_val_fold)
    val_predicted_labels = np.argmax(val_predictions, axis=1)
    val_true_labels = np.argmax(y_val_fold, axis=1)

    # Append predictions and true labels to the lists
    all_predictions.extend(val_predicted_labels)
    all_true_labels.extend(val_true_labels)


# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test_scaled, y_test_categorical)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

test_predictions = model.predict(x_test_scaled)
test_predicted_labels = np.argmax(test_predictions, axis=1)
test_true_labels = np.argmax(y_test_categorical, axis=1)

# Calculate the confusion matrix
cm_test = confusion_matrix(test_true_labels, test_predicted_labels)
print('Confusion Matrix (Test):')
print(cm_test)

# Calculate the classification report
class_report_test = classification_report(test_true_labels, test_predicted_labels, target_names=['Control', 'Concussed'])
print('Classification Report (Test):')
print(class_report_test)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report


y_pred_probs = model.predict(x_test_scaled)

y_pred = np.argmax(y_pred_probs, axis=1)

y_test_single = np.argmax(y_test_categorical, axis=1)

cm = confusion_matrix(y_test_single, y_pred)

report = classification_report(y_test_single, y_pred)

print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(report)


In [ ]:
TP = 3
FN = 3
FP = 1
TN = 9

Precision = TP / (TP + FP)
Sensitivity = TP / (TP + FN)
Specificity = TN /(TN + FP)
False_Pos_Rate = FP / (TN + FP)
False_Neg_Rate = FN / (FN + TP)
F1_Score = (2*Precision*Sensitivity) / (Precision + Sensitivity)
Accuracy = (TP+TN) / (TP+FP+TN+FN)

precision = "{:.2f}".format(Precision)
sensitivity = "{:.2f}".format(Sensitivity)
specificity = "{:.2f}".format(Specificity)
false_pos_rate = "{:.2f}".format(False_Pos_Rate)
false_neg_rate = "{:.2f}".format(False_Neg_Rate)
f1_score = "{:.2f}".format(F1_Score)
accuracy = "{:.2f}".format(Accuracy)

print("Precision          :", precision)
print("Sensitivity        :", sensitivity)
print("Specificity        :", specificity)
print("F1-Score           :", f1_score)
print("False Positive Rate:", false_pos_rate)
print("False Negative Rate:", false_neg_rate)
print("Accuraccy          :", accuracy)

In [ ]:
from tabulate import tabulate

data = [["Precision", 0.75], 
        ["Sensitivity", 0.50], 
        ["Specificity", 0.90], 
        ["F1-Score", 0.60],
        ["False Positive Rate", 0.10],
        ["False Negative Rate", 0.50],
        ["Accuraccy",0.75 ]]

col_names = ["Performance Metrics", "Result"]
print(tabulate(data, headers=col_names, tablefmt="fancy_grid"))
  
     


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


group_percentages = ["{0:.2%}".format(value) for value in
                     cm.flatten()/np.sum(cm)]

group_counts = ["{0:0.0f}".format(value) for value in
                cm.flatten()]

labels = [f"{v1}\n{v2}" for v1, v2 in
          zip(group_counts,group_percentages)]
          
labels = np.asarray(labels).reshape(2,2)

ax = sns.heatmap(cm, fmt = '' ,annot=labels, cmap = 'Greens')
ax.set_title('Confusion Matrix\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('\nActual Values\n');

ax.xaxis.set_ticklabels(['Control','Concussed'])
ax.yaxis.set_ticklabels(['Control','Concussed'])

plt.show()
     
